In [1]:
import cv2
import os
import base64
import requests
import pickle
import numpy as np

In [71]:
# pip install tensorflow

In [72]:
import tensorflow as tf
from tensorflow import keras

In [73]:
url = "http://localhost:8080/api/gethogfood"

In [74]:
path = r'Foods_dataset\training'
dir_list = os.listdir(path)
for file in dir_list:
    print(file)

Bread
Dairy product
Dessert
Egg
Fried food
Meat
Noodles-Pasta
Rice
Seafood
Soup
Vegetable-Fruit


In [75]:
classF = ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit']

In [76]:
def imgf2HOG(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url,json={"img":data})

    return response.json()

In [77]:
# read data food
def readDataFood(path):
    response = []
    for sub in os.listdir(path):
        for fn in os.listdir(path + '/' + sub):
            imgf_file_name = path + '/' + sub + "/" + fn
            img = cv2.imread(imgf_file_name)
            res = imgf2HOG(img)
            hopfood = list(res["hog"])
            hopfood.append(classF.index(sub))
            response.append(hopfood)
    return response

In [78]:
def savePkl(filename ,path):
    foods = readDataFood(path)
    write_path = filename + ".pkl"
    pickle.dump(foods, open(write_path,"wb"))
    print("Data Preparation is Done!!!")

In [79]:
# load data food to pkl
def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [80]:
#เรียกใช้ function savePkl file Trian Food
train_food = r'Foods_dataset\evaluation'
savePkl('train_foods',train_food)

Data Preparation is Done!!!


In [81]:
#เรียกใช้ function savePkl file Test Food
test_food = r'Foods_dataset\validation'
savePkl('test_foods',test_food)

Data Preparation is Done!!!


In [82]:
dataset_train_foods = loadPkl('train_foods')
print("Data train : ",len(dataset_train_foods))
dataset_test_foods = loadPkl('test_foods')
print("Data test : ",len(dataset_test_foods))

Data train :  3347
Data test :  3430


In [83]:
# แปลง list ข้อมูล train foods ให้เป็น array
train_food_arr = np.array(dataset_train_foods)
x_train = train_food_arr[:,0:-1]
y_train = train_food_arr[:,-1]

In [84]:
# แปลง list ข้อมูล test foods ให้เป็น array
test_food_arr = np.array(dataset_test_foods)
x_test = test_food_arr[:,0:-1]
y_test = test_food_arr[:,-1]

In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [86]:
# pip install scikit-learn

In [87]:
# เทรนข้อมูลด้วย Decision Tree
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [88]:
# test data
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 23.03206997084548
Confusion Matrix : 
 [[100   9  43  42  23  58   4  10  21  25  27]
 [  7  11  36   9  19  16   9   2   9  11  15]
 [ 43  18 129  45  36  61  20   8  42  67  31]
 [ 37  17  45  37  29  53  13  10  28  44  14]
 [ 34  13  51  21  55  48  10   4  41  26  23]
 [ 54  18  44  43  28 120  20  19  32  51  20]
 [ 15   4  22  14  11  24  12   3  10  25   7]
 [ 17   2   6   7   7  16   6   9   6  11   9]
 [ 41   4  42  41  23  34  11  13  74  35  29]
 [ 37  18  56  48  41  54  15  12  20 186  13]
 [ 25   7  30  18  23  14  12  11  21  14  57]]


In [ ]:
# save model file ด้วย pickle
path_model ='foodsmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))